In [30]:
# !pip install git+https://github.com/boudinfl/pke.git
# !pip install datasets
# !pip install ipywidgets
# !pip install nltk
# !python -m spacy download fr
# !pip install keybert
# !pip install sentence-transformers
import spacy
import pke
import pandas as pd
nlp = spacy.load("fr_core_news_sm")

NB_SAMPLES = 100 # Between 0 and 100 for WikiNews

In [31]:
import os
import time as time
from IPython.display import display
import string

def load_keyword_extraction_dataset(dataset_path,min_keys_count=False):
    if min_keys_count:
        min_keys = 10
    res = {
        "docs": [],
        "df": [],
        "keys": []
    } 
    os.makedirs(f"{dataset_path}/df", exist_ok=True)
    file_list = os.listdir(f"{dataset_path}/docsutf8")
    for filename in file_list:
        if not filename.endswith(".txt"): continue
        with open(f"{dataset_path}/docsutf8/{filename}", "r") as f:
            text = f.read()
            res["docs"].append(text.encode(errors="ignore").decode())
        keys_name = filename.replace(".txt", ".key")
        with open(f"{dataset_path}/keys/{keys_name}", "r") as f:
            text = f.read()
            doc = text.encode(errors="ignore").decode('utf-8', 'ignore')
            keys = doc.split("\n")[:-1] # La dernière ligne est vide, on la supprime
            if min_keys_count:
                min_keys = min(min_keys, len(keys)) # Permet de récupérer le nombre minimum de mots-clés associé à un document
            res["keys"].append(keys)
            
        # On calcule les fréquences de mots par document si elles n'existent pas
        df_name = filename.replace(".txt", ".tsv.gz") 
        if not os.path.isfile(f"{dataset_path}/df/{df_name}"):
            pke.utils.compute_document_frequency([res["docs"][-1]],
                           output_file=f"{dataset_path}/df/{df_name}",
                           language='fr')
        df = pke.load_document_frequency_file(input_file=f"{dataset_path}/df/{df_name}")
        res["df"].append(df)
        
    if min_keys_count:
        return res, min_keys
    return res

def strip_punctuation(s):
    translator = str.maketrans('', '', string.punctuation.replace('-','').replace("'",'').replace('’',''))
    result_string = s.translate(translator)
    return result_string

def remove_breaks(s):
    return s.replace('\n',' ').replace('\t',' ')

def remove_double_spaces(s):
    while '  ' in s:
        s = s.replace('  ',' ')
    return s

def preprocess_texts(lst_docs):
    lst_docs = [strip_punctuation(doc) for doc in lst_docs]
    lst_docs = [remove_breaks(doc) for doc in lst_docs]
    lst_docs = [remove_double_spaces(doc) for doc in lst_docs]
    lst_docs = [doc.lower() for doc in lst_docs]
    return lst_docs
    

In [32]:
dataset, min_keys = load_keyword_extraction_dataset('WKC',min_keys_count=True)
list_docs = preprocess_texts(dataset['docs'][:NB_SAMPLES])
list_df = dataset['df'][:NB_SAMPLES]
list_keys = dataset['keys'][:NB_SAMPLES]

In [33]:
from pke.unsupervised import MultipartiteRank, YAKE, TfIdf

outputs = {}
times = {}
for model in [YAKE, TfIdf, MultipartiteRank]: # On utilise les trois modèles pour extraire des mots-clés
    outputs[model.__name__] = []
    extractor = model()
    current_time = time.time() # On mesure le temps d'exécution
    for doc, df in zip(list_docs, list_df):
        extractor.load_document(input=doc, language='fr')
        extractor.grammar_selection(grammar="NP: {<ADJ>*<NOUN|PROPN>+}")  # Noun phrase selection based on grammar rules
        if model.__name__ == "TfIdf":
            extractor.candidate_weighting(df=df)
        else:
            extractor.candidate_weighting()
            
        outputs[model.__name__].append([u for u,v in extractor.get_n_best(n=min_keys,redundancy_removal=True)]) # On récupère les mots-clés, en ne gardant que le nombre minimum de mots-clés attendus
    times[model.__name__] = time.time() - current_time # On enregistre le temps d'exécution

In [34]:
from keybert import KeyBERT

outputs["KeyBERT"] = []
kb_model = KeyBERT(model='distiluse-base-multilingual-cased-v1')
current_time = time.time() # On mesure le temps d'exécution
for key, doc in enumerate(list_docs):
    doc_embeddings, word_embeddings = kb_model.extract_embeddings(doc)
    keywords = kb_model.extract_keywords(doc, top_n=min_keys, doc_embeddings=doc_embeddings, word_embeddings=word_embeddings)
    outputs["KeyBERT"].append([u for u,v in keywords]) # On récupère les mots-clés, en ne gardant que le nombre minimum de mots-clés attendus
times["KeyBERT"] = time.time() - current_time # On enregistre le temps d'exécution

In [35]:
pd.DataFrame(outputs).head()

,YAKE,TfIdf,MultipartiteRank,KeyBERT
0,"[42e festival, 42e édition, défilés journalier...","[jusqu'au, dan, autres animations récurrentes,...","[lorient, août, 42e festival, grande nuit, aca...","[festival, interceltique, célébration, interce..."
1,"[poste jean-paul bailly, poste direction, dern...","[noyal-sur-vilaine ille-et-vilaine, domicile c...","[syndicat, postier, suicide, poste direction, ...","[suicide, décès, mort, morte, courrier, privé]"
2,"[gros fraudeurs croatie publication, gros frau...","[croatie publication, gros fraudeurs croatie p...","[noms, croatie publication, associations, gros...","[croatie, croates, fraudeurs, impôts, transpar..."
3,"[high-tech annonce, nouvel appareil, second st...","[h, mm, high-tech annonce, « grand format, jus...","[h, mm, jeux, dimensions, 3ds xl, high-tech an...","[3ds, nintendo, console, ds, dsi, japon]"
4,"[60 -, ria novosti valentine, scientifiques ru...","[60 -, v, mêmes résultats, seule zone, «, scie...","[scientifiques russes, yéti, poils, créature, ...","[russes, russie, scientifiques, scientifique, ..."


# Benchmarking

In [36]:
# !pip install rouge

In [37]:
from rouge import Rouge
rouge = Rouge() # On utilise la librairie rouge pour comparer les mots-clés extraits avec les mots-clés attendus

In [38]:
for i in range(0,len(list_keys)):
    list_keys[i] = list_keys[i][:min_keys] # On ne garde que le nombre minimum de mots-clés attendus, pour avoir le même nombre de mots-clés attendus et extraits

In [39]:
scores_rouges = {}
for model in outputs:
    scores_rouges[model] = []
    for i in range(0,len(list_keys)):
        scores_rouges[model].append(rouge.get_scores(outputs[model][i], list_keys[i],avg=True)) # On calcule les scores ROUGE pour chaque document, avec rouge-1, rouge-2 et rouge-l

In [40]:
display(pd.DataFrame(scores_rouges).head())
pd.DataFrame(scores_rouges).to_csv('scores_rouges.csv',index=False) # On enregistre les scores ROUGE dans un fichier CSV, pour pouvoir les réutiliser plus tard au besoin

,YAKE,TfIdf,MultipartiteRank,KeyBERT
0,"{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'r...","{'rouge-1': {'r': 0.16666666666666666, 'p': 0....","{'rouge-1': {'r': 0.041666666666666664, 'p': 0...","{'rouge-1': {'r': 0.041666666666666664, 'p': 0..."
1,"{'rouge-1': {'r': 0.08333333333333333, 'p': 0....","{'rouge-1': {'r': 0.05555555555555555, 'p': 0....","{'rouge-1': {'r': 0.3333333333333333, 'p': 0.3...","{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'r..."
2,"{'rouge-1': {'r': 0.3333333333333333, 'p': 0.1...","{'rouge-1': {'r': 0.16666666666666666, 'p': 0....","{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'r...","{'rouge-1': {'r': 0.3333333333333333, 'p': 0.3..."
3,"{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'r...","{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'r...","{'rouge-1': {'r': 0.3333333333333333, 'p': 0.2...","{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'r..."
4,"{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'r...","{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'r...","{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'r...","{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'r..."


+ Expliquez le fonctionnement de chaque algorithme que vous avez choisi / utilisé
+ Quel algorithme a obtenu le meilleur score ROUGE ?

In [41]:
scores_rouges_mean = {}

def add(scores, param): # Fonction pour additionner les scores ROUGE de deux documents
    for i in ['rouge-1','rouge-2','rouge-l']:
        for j in ['f','p','r']:
            scores[i][j] += param[i][j]
    return scores

for model in scores_rouges: # On calcule les scores ROUGE moyens pour chaque modèle
    scores = {'rouge-1': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.0, 'p': 0.0, 'r': 0.0}}
    for i in range(0,len(scores_rouges[model])):
        scores = add(scores,scores_rouges[model][i]) # On additionne les scores ROUGE de chaque document
    for i in ['rouge-1','rouge-2','rouge-l']:
        for j in ['f','p','r']:
            scores[i][j] = round(scores[i][j]/len(scores_rouges[model]),3)  # On divise les scores ROUGE par le nombre de documents pour obtenir la moyenne
    scores['global'] = { # On calcule la moyenne des scores ROUGE-1, ROUGE-2 et ROUGE-L, pour faciliter la comparaison entre les modèles
        'f': round((scores['rouge-1']['f'] + scores['rouge-2']['f'] + scores['rouge-l']['f'])/3,3),
        'p': round((scores['rouge-1']['p'] + scores['rouge-2']['p'] + scores['rouge-l']['p'])/3,3),
        'r': round((scores['rouge-1']['r'] + scores['rouge-2']['r'] + scores['rouge-l']['r'])/3,3)
    }
    scores_rouges_mean[model] = scores
pd.DataFrame(scores_rouges_mean)

,YAKE,TfIdf,MultipartiteRank,KeyBERT
rouge-1,"{'f': 0.042, 'p': 0.042, 'r': 0.052}","{'f': 0.033, 'p': 0.032, 'r': 0.038}","{'f': 0.068, 'p': 0.083, 'r': 0.068}","{'f': 0.045, 'p': 0.057, 'r': 0.04}"
rouge-2,"{'f': 0.002, 'p': 0.002, 'r': 0.002}","{'f': 0.002, 'p': 0.003, 'r': 0.001}","{'f': 0.002, 'p': 0.002, 'r': 0.002}","{'f': 0.0, 'p': 0.0, 'r': 0.0}"
rouge-l,"{'f': 0.042, 'p': 0.042, 'r': 0.052}","{'f': 0.033, 'p': 0.032, 'r': 0.038}","{'f': 0.068, 'p': 0.083, 'r': 0.068}","{'f': 0.045, 'p': 0.057, 'r': 0.04}"
global,"{'f': 0.029, 'p': 0.029, 'r': 0.035}","{'f': 0.023, 'p': 0.022, 'r': 0.026}","{'f': 0.046, 'p': 0.056, 'r': 0.046}","{'f': 0.03, 'p': 0.038, 'r': 0.027}"


+ Les mots-clés extraits semblent-ils raisonnables si vous évaluez vous-même un petit
ensemble d'entre eux ?

In [42]:
sample_indexes = pd.DataFrame(outputs).sample(10).index.array # On sélectionne 10 index au hasard, pour évaluer les mots-clés extraits manuellement
outputs_samples = {}
for i in outputs:
    outputs_samples[i] = []
    for j in sample_indexes:
        outputs_samples[i].append(outputs[i][j])
outputs_samples = pd.DataFrame(outputs_samples) # On récupère les mots-clés extraits correspondant aux index sélectionnés
list_docs_samples = pd.DataFrame([list_docs[i] for i in sample_indexes]) # On récupère les documents correspondant aux index sélectionnés
list_keys_samples = pd.DataFrame([list_keys[i] for i in sample_indexes]) # On récupère les mots-clés attendus correspondant aux index sélectionnés
display(list_keys_samples)
display(outputs_samples)
display(list_docs_samples)

,0,1,2,3,4,5
0,plus riches,plus pauvres,pauvres,crise financière,france,niveau de vie
1,ambassade,agressée sexuellement,correspondante,audiovisuel,journaliste de france 24,sonia dridi
2,nouvelle particule,matière,cern,découverte,physique,particule
3,banques belges,moyen de paiement très sécurisé,sites de commerce en ligne,piratage,perte,parquet fédéral
4,naissance,codes,fêtent,grande-bretagne,célébration,université de manchester
5,discipline,catalan,député-maire,linguistique,décision,député
6,déficit,impôts,taxe,fiscalité,hausse,pierre mosvovici
7,antisémitisme,agression antisémite,toulouse,actes d'une extrême violence,lycéen,agression
8,législative,purger,pp,gouvernement rajoy,journalisme,Ana Pastor
9,planète,télescope spatial kepler,système à quatre étoiles,système stellaire quadruple,découverte,diamants


,YAKE,TfIdf,MultipartiteRank,KeyBERT
0,"[riches france, fourchette 2002 - 2004, nouvel...","[riches france, c'est, fourchette 2002 - 2004,...","[crise, mise, niveau, ans, revenu, vie]","[crise, pauvreté, financière, pauvres, 2008, 2..."
1,"[violente agression, télévision france, vérita...","[autres témoins, j'ai, x, autre part, l’ ambas...","[france, violente agression, chaîne, caire, jo...","[égypte, égyptien, egypte, agression, agressio..."
2,[facilities council britannique john womersley...,"[science découverte, higgs science découverte,...","[boson, nouvelle particule, higgs, découverte,...","[higgs, boson, particules, atomique, scientifi..."
3,"[grandes banques, sécurité c’est, fédéral anno...","[«, est, sécurité c’est, l’ affaire, tous c’es...","[banques, euros, fédération, millions, parquet...","[hackers, piratage, banques, pirates, bancaire..."
4,"[turing informatique célébration, informatique...","[informatique célébration, turing informatique...","[alan, informatique célébration, centenaire, n...","[informatique, informaticiens, hacker, computi..."
5,"[langue catalane, îles baléares, populaire esp...","[populaire espagnol1, «, pp, qu'antoni pastor,...","[langue catalane, parti, député, îles baléares...","[catalan, catalane, parlementaire, îles, lingu..."
6,"[finances pierre moscovici, gouvernement compt...","[retour, qu'il, finances pierre moscovici, « c...","[euros, milliards, impôts, entreprises, gouver...","[fiscales, fiscalité, impôts, impôt, taxe, 2012]"
7,"[lyon france agression, france agression, nouv...","[«, france agression, lyon france agression, «...","[france agression, lyon, « signe religieux, mi...","[antisémitisme, antisémites, antisémite, lyon,..."
8,"[dernier pastor, vrai journalisme, maría dolor...","[pp, «, autre changement, premier temps, même ...","[pastor, rtve, « telenotícies, journaliste, ra...","[rajoy, rubalcaba, purge, espagnole, socialist..."
9,"[toute première fois, amateurs américains kian...","[espace découverte, toute première fois, amate...","[soleils, système, étoiles, exoplanète, astron...","[astronomes, planethuntersorg, planète, exopla..."


,0
0,france la crise financière touche les plus pa...
1,égypte une journaliste de france 24 victime d...
2,science découverte du boson de higgs science ...
3,banques belges piratées « le parquet exagère ...
4,informatique célébration du centenaire de la ...
5,îles baléares le parti populaire exclut le dé...
6,france les impôts augmenteront de 72 milliard...
7,france agression antisémite contre un lycéen ...
8,espagne purge à la rtve après l'arrivée de ra...
9,espace découverte d'une exoplanète possédant ...


+ Combien de temps a-t-il fallu à chaque algorithme pour extraire les KP du dataset ?

Les temps des algorithmes sont les suivants :
TfIdf : 90 secondes
YAKE : 89 secondes
MultipartiteRank : 95 secondes

In [43]:
times # times contient les temps d'exécution des algorithmes sur l'ensemble des documents

{'YAKE': 98.6207640171051,
 'TfIdf': 95.64659881591797,
 'MultipartiteRank': 97.64895558357239,
 'KeyBERT': 5.224766492843628}

+ Pourquoi pensez-vous qu'il a le mieux fonctionné ?
+ Comment représenteriez-vous chaque document et ses phrases-clés extraites respectives
sous la forme d'un graphe de connaissances ? Quels vocabulaires utiliseriez-vous ?